You can read the paper for more detail. <BR>
Reference: https://ieeexplore.ieee.org/abstract/document/5197422

In [1]:
class Matrix_Factorization:
    def __init__(self, alpha = 0.00001, iterations = 50, num_of_latent = 200, lam = 0.0005):
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 0.01
                        num_iter: Number of Iterations to update coefficient with training data
                        num_of_latent: Number of latent factor.
                        lam: Regularization constant
                        
            
            TODO: 1. Initialize all variables needed.
        """
        
        self.alpha = alpha
        self.iterations = iterations
        self.num_of_latent = num_of_latent
        self.lam = lam
        
    def fit(self, train):
        """
            Train: list of tuples with (User, Movie, Rating)
            num_user: Number of unique user.
            num_movie: Number of unique movie
            
            TODO: 2. Initialize num_user and num_movie
                  3. Save the training set.
                  4. Initialize P and Q matrix, with normal distribution with mean = 0. 
                  Hint: Think about what P and Q represent, what they should do.Think about the shape too. 
            
        
        """
        num_user = max(map(lambda t:t[0], train)) + 1
        num_movie =  max(map(lambda t:t[1], train)) + 1
        
        self.train = train
        
        self.P = np.random.normal(loc = 0, size=[num_user, self.num_of_latent])
        self.Q = np.random.normal(loc = 0, size=[num_movie, self.num_of_latent])
        
        rmse_lst = []
        
        """
            TODO: 5: Calculate the error, using P and Q matrix. 
                  6: We need to check if the absolute value error is less than some constant. Store the previous Q and P for adaptive learning rate.
                      If it is less than that constant then we update P and Q matrix. 
                      (When update, update the P and Q at the same time. Think about why it is important.)
                      Otherwise use the error to update the Q and P matrix.
                      
                  7: For each entry update temp_mse, and append the Current iteration RMSE to rmse_lst.
                  
        """
 
        for f in range(self.iterations):
            ### Random Shuffle. Why is this called?
            np.random.shuffle(self.train)
            
            temp_mse = 0

            previous_Q = self.Q.copy()
            previous_P = self.P.copy()
            
            Count = 0
   
            for tup in self.train:
                u,i,rating = tup
                error = rating - self.Q[i].T @ self.P[u]
                
                if abs(error) > 20 :
                    continue
                Count += 1
                temp_mse += error**2
                
                #### Don't Modify this code, helpful for converge.
#                 if np.isinf(self.Q).any() or np.isinf(self.P).any() or np.isnan(self.Q).any() or np.isnan(self.P).any():
#                     pass
#                 else:
                    #### NEED TO MODIFY #### Update P and Q.
                self.P[u] += 2 * self.alpha * error * self.Q[i] - self.lam * self.P[u]
                self.Q[i] += 2 * self.alpha * error * self.P[u] - self.lam * self.Q[i]
                
            rmse_lst.append(temp_mse / Count)
            print(f, rmse_lst[-1])
            
            
            """
                TODO: 8: Implement the adaptive learning rate.
                         If current rmse is less than previous iteration, let's increase by a factor range from 1 - 1.5
                         Otherwise we decrease by a factor range from 0.5 - 1
                      9: If the current rmse is greater than previous iteration.
                         Check the relative error, (previous - current)/ previous.
                         If it is greater than 0.1, we restore the previous Q and P. (Try without it. Think about why we need this.)
            """
            if len(rmse_lst)==1 or rmse_lst[-1]<rmse_lst[-2]:
                self.alpha = self.alpha*1.25
            else:
                self.alpha = self.alpha*0.75
                if (rmse_lst[-1] - rmse_lst[-2])/rmse_lst[-2] > 0.1:
                    self.Q = previous_Q
                    self.P = previous_P
                    
                
        self.rmse = rmse_lst
        
    def ind_predict(self, tup):
        """
            tup: One single entry, (user, movie)
            
            TODO: 10: Use P and Q to make prediction on single entry.
            
        """
        u,i = tup
        
        return self.Q[i].T @ self.P[u]
    def predict(self, X):
        """
            X: list of entries
            
            TODO: 11: Use ind_predict we create to make predicitons.
        """
        res = []
        for i in X:
            res.append(self.ind_predict(i))
        return res

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ratings = pd.read_csv('./ratings.csv')
ratings = ratings.sort_values('timestamp').reset_index(drop = True).astype(int)
a = np.sort(ratings.movieId.unique())
index_dict = {}
for i in range(len(a)):
    index_dict[a[i]]  = i
ratings.movieId = ratings.movieId.apply(lambda x: index_dict[x]) 

grouped_userId = list(ratings.groupby(['userId']).apply(lambda x: x[['movieId','rating']].values))
grouped_movieId = list(ratings.groupby(['movieId']).apply(lambda x: x[['userId','rating']].values))

In [4]:
np.random.seed(0)
train = set()
test = set()

for i in range(len(grouped_movieId)):
    temp_len = len(grouped_movieId[i])
    for j in range(temp_len):
        if j < temp_len * 0.8:
            train.add((grouped_movieId[i][j][0]-1,i - 1, grouped_movieId[i][j][1]))
        else:
            test.add((grouped_movieId[i][j][0] -1, i - 1,grouped_movieId[i][j][1]))
train = list(train)
test = list(test)

In [5]:
len(train), len(test)

(84071, 16765)

#### Try to understand what is done for data cleaning. 

#### If you are doing things correctly (with tuning), you will get something lower than 1.1 test RMSE.

In [20]:
clf = Matrix_Factorization(alpha = 0.00001, iterations = 100, num_of_latent = 200, lam = 0.001)
clf.fit(train)

0 86.48621568136389
1 60.23648236911481
2 46.22261163974643
3 37.41356128338474
4 31.210382677873717
5 26.826540482113412
6 23.551035154807597
7 20.98983135060455
8 18.99522337325284
9 17.420100649192683
10 16.109918624348058
11 15.016548357447942
12 14.09967332435114
13 13.313424753789674
14 12.645811547942074
15 12.032159876336042
16 11.457702862514447
17 10.907553731515899
18 10.383743274381247
19 9.75031096030124
20 8.85774601446156
21 7.471321605818507
22 5.700545633486387
23 4.232672060316316
24 3.314123208867258
25 2.777918932322361
26 2.496243262493866
27 2.3611443110053454
28 2.4706523573931922
29 1.6377822780776694
30 1.5369443212159497
31 2.0725272581261507
32 1.2974535175593742
33 1.5302625351435895
34 1.054098038276624
35 1.1846876156305588
36 0.8932719117635987
37 0.9900178661424461
38 0.8130132408020676
39 0.8644259807231988
40 0.6903700281042745
41 0.7027488237180819
42 0.6135487474409538
43 0.6055089383392284
44 0.6523452943648825
45 0.5432557252094005
46 0.55068569056

In [21]:
error = 0
pred = clf.predict(map(lambda t: (t[0],t[1]),test))
for i in range(len(test)):
    error += (test[i][2] - pred[i])**2

In [22]:
(error/ len(test))** (0.5)

2.770206121665545

In [ ]:
class Matrix_Factorization_with_bias:
    def __init__(self, alpha = 0.00001, iterations = 50, num_of_latent = 200, lam = 0.01):
        
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 0.01
                        num_iter: Number of Iterations to update coefficient with training data
                        num_of_latent: Number of latent factor.
                        lam: Regularization constant
 
            
            TODO: 1. Initialize all variables needed.
        """
            
        self.alpha = alpha
        self.iterations = iterations
        self.num_of_latent = num_of_latent
        self.lam = lam


        
    def fit(self, train):
        """
            Train: list of tuples with (User, Movie, Rating)
            num_user: Number of unique user.
            num_movie: Number of unique movie
            
            TODO: 2. Initialize num_user and num_movie.
                  3. Save the training set.
                  4. Initialize bu , bi and b. b is the global mean of the rating.
                  5. Initialize P and Q matrix. 
                  Hint: Think about what P and Q represent, what they should do.Think about the shape too. 
                  
            
        
        """
        
        num_user, num_movie = train.shape

        self.train = train
        
        self.P = [1]
        self.Q = [1]
        
        self.bu = [1]
        self.bi = [1]
        self.b = 2 
        
        rmse_lst = []
        
        """
            TODO: 5: Calculate the error, using P , Q , bu , bi and b. 
                  6: Update the P , Q , bu , bi and b with error you calculate. 
                    (Think about why we don't need to check the absolute of error)
                  7: For each entry update temp_mse, and append the Current iteration RMSE to rmse_lst.
                  
        """


        for f in range(self.iterations):
            
            np.random.shuffle(self.train)
            
            temp_mse = 0
            previous_Q = [1]
            previous_P = [1]
            previous_bu = [1]
            previous_bi = [1]
            
            Count = 0
            for tup in self.train:
                u,i,rating = tup
                
                error = 1
            

                Count += 1   
                temp_mse += 1
                
                bu = [1]
                bi = [1]
                
                temp_Q = [1]
                
                Q = [1]
                P = [1]
                
                # You might want to update column by column
                self.Q = Q
                self.P = P
                self.bu = bu
                self.bi = bi
            
            rmse_lst.append(1)
                
            
            
            """
                TODO: 8: Implement the adaptive learning rate.
                         If current rmse is less than previous iteration, let's increase by a factor range from 1 - 1.5
                         Otherwise we decrease by a factor range from 0.5 - 1
                      9: If the current rmse is greater than previous iteration.
                         Check the relative error, (previous - current)/ previous.
                         If it is greater than 0.1, we restore the previous Q and P. (Try without it. Think about why we need this.)
            """
                
            
            if True:
                self.alpha = 1
            else:
                self.alpha = 1
                if True :
                    self.Q = [1]
                    self.P = [1]
                    self.bu = [1]
                    self.bi = [1]
                    
                    
            
            
        self.rmse = rmse_lst
            
    def ind_predict(self, tup):
        """
            tup: One single entry, (user, movie)
            
            TODO: 10: Use P and Q to make prediction on single entry.
            
        """
        u,i = tup
        return 1

    
    def predict(self, X):
        """
            X: list of entries
            
            TODO: 11: Use ind_predict we create to make predicitons.
        """
        res = []
        for i in X:
            res.append(1)
        return res
            
        
    

### If you are doing thing correctly (With tuning), you should get a Test RMSE below 1.

In [ ]:
%%time
clf = Matrix_Factorization_with_bias()
clf.fit(train)

In [ ]:
error = 0
pred = clf.predict(test)
for i in range(len(test)):
    error += (test[i][2] - pred[i])**2

In [ ]:
(error/ len(test))** (0.5)